In [1]:
import pymorphy2

from spacy.lang import ru
from nltk.stem.snowball import SnowballStemmer 
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop = set(nltk.corpus.stopwords.words('russian'))
Unused_chars=[':',',','.','-','\n','?','!']
stemmer = SnowballStemmer('russian')
morph = pymorphy2.MorphAnalyzer()


def tokenize(file_text):
    tokens = word_tokenize(file_text)
    tokens = [i for i in tokens if (i not in stop)]
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]
    return tokens


def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [get_lemma(token) for token in tokens]
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    

#def get_lemma2(word):
 #   return WordNetLemmatizer().lemmatize(word)




[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
import pymorphy2

decision_value=0.75



#Логический разделитель
Logic_Sep=['PREP','CONJ','PRCL']
Logic_AdVerb=['PRTF','PRTS','GRND']
Logic_Adj=['ADJF','ADJS']
verb=['VERB','INFN']
Separators=Logic_Sep+Logic_AdVerb+verb+Logic_Adj

def logic_construct_extractor(string):
    Main_noun=''
    verbs=[]
    temp_string=''
    state=False
    Logical_structure=[]
    for word in string.split():
        f_word=word
        for char in Unused_chars:
            if char in f_word:
                f_word=f_word.replace(char,'')
        p=morph.parse(f_word)[0]
        if p.tag.POS in Separators and len(temp_string)>0 and not state:
            Logical_structure.append(temp_string)
            temp_string=f_word+' '
            state=True
        else:
            temp_string+=f_word+' '
            state=False
    Logical_structure.append(temp_string)
    return Logical_structure



def nearest_words(filename,seek_word):
    s_word=prepare_text_for_lda(seek_word)[0]
    full_text=[]
    result=[]
    with open('Lyrics\\'+filename+'.txt','r',encoding='utf-8') as f:
        for line in f:
            tokens = prepare_text_for_lda(line)
            full_text.append(tokens)
    for line in full_text:
        prev_word=''
        need_next=False
        for token in line:
            if need_next:
                result.append(token)
                need_next=False
            if token==seek_word:
                result.append(prev_word)
                need_next=True
            prev_word=token
    return result
                
def nearest_logic_words(filename,seek_word):
    s_word=prepare_text_for_lda(seek_word)[0]
    full_text=[]
    result=[]
    with open('Lyrics\\'+filename+'.txt','r',encoding='utf-8') as f:
        for line in f:
            if not len(line)>1:
                continue
            for string in line.split('.'):
                logic_structs=logic_construct_extractor(string)
                for logic in logic_structs:
                    if seek_word in logic:
                        result.append(logic)
    return result   
    
    
    
#all_words=nearest_words('Chapter2','розовый')
all_words=nearest_logic_words('Chapter2','розовый')
print(all_words)

['розовый запах ', 'жирный розовый дух ']


In [7]:
custom_useless_words=['из-за','марка','ведь','патриарший','борменталь','филипп','кихот','санчо','антоний','сансон','шарик','мой','такой','другой','ваш','этот','сам','свой','тот','варенуха','степ','маргарита','иван','воланд','коровьев','пилат','николай','какой-то','свой','самый','быть','стать','это','что-то','весь','какой','мочь','никанор','который','ты','никакой','азазело','азазелло','берлиоз']
count=0
with open('Lyrics\sample.txt','r',encoding='utf-8') as f:
    for line in f:
        if len(line)<4:
            continue
        if count>1:
            break
        print(line)
        print(line.split())
        tokens=prepare_text_for_lda(line)
        print(tokens)
        count+=1
        


﻿Секретарь смертельно побледнел и уронил свиток на пол.
['\ufeffСекретарь', 'смертельно', 'побледнел', 'и', 'уронил', 'свиток', 'на', 'пол.']
['\ufeffсекретарить', 'смертельно', 'побледнеть', 'уронить', 'свитка']


In [6]:
#Стандарные символы
import glob
symbols=[]
for filename in glob.glob('Symbols\*.txt'):
    line=''
    filename=filename.replace('Symbols\\','')
    filename=filename.replace('.txt','')
    for word in filename.split('_'):
        line+=word+' '
    tokens = prepare_text_for_lda(line)
    for token in tokens:
        symbols.append(token)

In [7]:
with open('symbols.txt','w',encoding='utf-8') as f:
    for symbol in symbols:
        f.write('{} \n'.format(symbol))
#print(symbols)
print('ready')

ready


In [12]:
#Символы, связанные с сонниками
from docx import Document
sonic_symbols=[]
document=Document('Sonics\\sonnikall.docx')
for p in document.paragraphs:
    for run in p.runs:
        count=0
        sonic_symbol=''
        #print(run.text)
        for word in run.text.split():
            if word==word.capitalize():
                count+=1
            if count>1:
                break
            tokens = prepare_text_for_lda(word)
            #print(tokens)
            if len(tokens)>0:
                sonic_symbol+=tokens[0]+' '
        if len(sonic_symbol)>0:
            sonic_symbols.append(sonic_symbol)
#with open('Sonics\\sonnikall.docx') as f:
    #document=Document(f)

In [13]:
sonic_symbols.pop(0)
sonic_symbols.pop(0)
sonic_symbols.pop(0)
print(sonic_symbols)

['аборт ', 'абрикос ', 'автомобиль ', 'судьбе.управлять ', 'машина предвещать потеря болезнь ', 'неудачи.просто ', 'увидеть автомобиль поездка перемена судьба приятный путешествиям.ести ', 'автор ', 'адам ', 'адвокат ', 'ого-то имущества.ести адвокат защищать обещать ', 'аккордеон ', 'акробат ', 'ответственность что-то трудность происки ваш врагов.женщина-акробатка ', 'актёр ', 'раскаянию.длить ', 'мужчина видеть развлекаться актриса означать ссора жена возлюбить принести несчастья.увидеть актёр-любитель сцена сулить скорый осуществление ', 'акула ', 'акушерка ', 'аллея ', 'алмаз ', 'алтарь ', 'альбом ', 'амбар ', 'америка ', 'ампутация ', 'ананас ', 'ангел ', 'ангина ', 'антилопа ', 'апельсин ', 'арфа ', 'атлас ', 'африка ', 'бабушка дедушка ', 'встретить ', 'ваш бабушка дедушка беседовать они ', 'балкон ', 'банан ', 'банк ', 'банкрот ', 'баня турецкий ', 'барабан ', 'барсук ', 'бархат ', 'бассейн ', 'башня ', 'бездна ', 'бедняк нищий ', 'бедро ', 'бежать ', 'подмочить репутация одино

# Все что выше - запустить один раз и все

In [113]:
temp1,temp2=read_file('Chapter1')
w_count=0
for line in temp1:
    for w in line.split():
        w_count+=1
print('Количество слов в Chapter1 = {}'.format(w_count))
temp1,temp2=read_file('Chapter2')
w_count=0
for line in temp1:
    for w in line.split():
        w_count+=1
print('Количество слов в Chapter2 = {}'.format(w_count))
temp1,temp2=read_file('Chapter3')
w_count=0
for line in temp1:
    for w in line.split():
        w_count+=1
print('Количество слов в Chapter3 = {}'.format(w_count))
temp1,temp2=read_file('Chapter13')
w_count=0
for line in temp1:
    for w in line.split():
        w_count+=1
print('Количество слов в Chapter13 = {}'.format(w_count))


Количество слов в Chapter1 = 3626
Количество слов в Chapter2 = 7064
Количество слов в Chapter3 = 1428
Количество слов в Chapter13 = 6371


In [3]:
import csv

#Возвращает лист словарей со следующими ключами
'''
#
Фрагмент
Суперслово/словосочетание
Характер слова
Значение
Дополнительно
'''
def csv_read(filename):
    with open('Data\\'+filename+'.csv') as csvfile:
        input_file1 = csv.DictReader(csvfile,delimiter=";")
        input_file=list(input_file1)
        result=[]
        for d in input_file:
            if d['Суперслово/словосочетание']!='' and d['Значение']!='':
                result.append(d)
            else:
                break
        return result
    
def symbol_in_csv(input_file,symbols):
    count=0
    found_symbols=[]
    limit=0
    for row in input_file:
        limit+=1
        #print('Обработано {} из {}'.format(limit,len(symbols)))
        line=row['Суперслово/словосочетание']
        tokens=prepare_text_for_lda(line)
        for symbol in symbols:
            if symbol in tokens:
                add_list={}
                count+=1
                add_list['символ']=symbol
                add_list['словосочетание']=row['Суперслово/словосочетание']
                add_list['фрагмент']=row['Фрагмент']
                add_list['значение']=row['Значение']
                found_symbols.append(add_list)
                break
    return found_symbols

def read_file(filename):
    Chapter2_tokenized=[]
    Chapter2_text=[]
    with open('Lyrics\\'+filename+'.txt',encoding='utf-8') as f:
        for line in f:
            Chapter2_text.append(line)
            tokens = prepare_text_for_lda(line)
            Chapter2_tokenized.append(tokens)
    return Chapter2_text,Chapter2_tokenized

def take_symbol_lines(Chapter_tokenized,Chapter_text):
    count=0
    symbol_lines=[]
    for tokens in Chapter_tokenized:
        for symbol in symbols:
            if symbol in tokens:
                symbol_lines.append(Chapter_text[count])
        count+=1
    #print(count)
    return symbol_lines


print('ready')

ready


In [111]:
#Первая стадия
count_symbol=0
count_intersection=0
count_symbol_in_text=0
count_csv=0

for full_filename in glob.glob('Data//*.csv'):
    filename=full_filename.replace('Data\\','')
    filename=filename.replace('.csv','')
    print('Обрабатывается {}'.format(filename))
    #Считываю файл csv и возвращаю лист словарей
    input_file=csv_read(filename)
    count_csv+=len(input_file)
    print('{} - количество символов в размеченном csv'.format(len(input_file)))
    #Ищу символы в считанном файле
    found_symbols=symbol_in_csv(input_file,symbols)
    #print('Поиск символов в {}'.format(filename))
    count_symbol+=len(found_symbols)
    print('{} - количество символов из словаря в размеченном csv в {}'.format(len(found_symbols),filename))
    Chapter_text,Chapter_tokenized=read_file(filename)
    symbol_lines=take_symbol_lines(Chapter_tokenized,Chapter_text)
    count_symbol_in_text+=len(symbol_lines)
    print('{} - количество символов из словаря в главе {}'.format(len(symbol_lines),filename))
    temp=[]
    for symbol in found_symbols:
        for symbol_line in symbol_lines:
            if symbol['словосочетание'] in symbol_line or symbol_line in symbol['словосочетание']:
                temp.append(symbol_line)
                break
    count_intersection+=len(temp)
    print('{} - количество совпадений выявленных символов в одном и том же предложении в {}'.format(len(temp),filename))
    print('\n\n\n\n')
    

Обрабатывается Chapter1
37 - количество символов в размеченном csv
21 - количество символов из словаря в размеченном csv в Chapter1
1115 - количество символов из словаря в главе Chapter1
10 - количество совпадений выявленных символов в одном и том же предложении в Chapter1





Обрабатывается Chapter13
32 - количество символов в размеченном csv
27 - количество символов из словаря в размеченном csv в Chapter13
1476 - количество символов из словаря в главе Chapter13
10 - количество совпадений выявленных символов в одном и том же предложении в Chapter13





Обрабатывается Chapter2
56 - количество символов в размеченном csv
46 - количество символов из словаря в размеченном csv в Chapter2
2153 - количество символов из словаря в главе Chapter2
24 - количество совпадений выявленных символов в одном и том же предложении в Chapter2





Обрабатывается Chapter3
22 - количество символов в размеченном csv
11 - количество символов из словаря в размеченном csv в Chapter3
301 - количество символов и

In [112]:
print('{} - сумма всех символов из словаря в размеченных csv'.format(count_symbol))
print('{} - сумма всех символов из словаря в 4 главах'.format(count_intersection))
print('{} - сумма всех совпадений выявленных символов в одном и том же предложении'.format(count_symbol_in_text))
print('{} - сумма всех символов в размеченных csv'.format(count_csv))

105 - сумма всех символов из словаря в размеченных csv
47 - сумма всех символов из словаря в 4 главах
5045 - сумма всех совпадений выявленных символов в одном и том же предложении
147 - сумма всех символов в размеченных csv


# Работа с именами

In [3]:
# Одиночные вхождения имени, фамилии, отчества
from natasha import NamesExtractor

text=''''''
#with open('Lyrics\Chapter3.txt',encoding='utf-8') as f:
with open('Lyrics\Master.txt',encoding='utf-8') as f:
    for line in f:
        text+=line

Names=[]
Families=[]
Lasts=[]
extractor = NamesExtractor()
matches = extractor(text)
for match in matches:
    if match.fact.first is not None:
        tokens=prepare_text_for_lda(match.fact.first)
        if len(tokens)>0:
            Names.append(tokens[0])
    if match.fact.last is not None:
        tokens=prepare_text_for_lda(match.fact.last)
        if len(tokens)>0:
            Families.append(tokens[0])
    if match.fact.middle is not None:
        tokens=prepare_text_for_lda(match.fact.middle)
        if len(tokens)>0:
            Lasts.append(tokens[0])


Names_set=set(Names)
Fam_set=set(Families)
Lasts_set=set(Lasts)

{'аристотель', 'тимофей', 'аннушка', 'иешуа', 'стёпа', 'дунчиль', 'роберт', 'иванов', 'фауст', 'гамаленькая', 'алоизий', 'вифаний', 'гарун', 'иоганн', 'аполлон', 'иммануил', 'марья', 'василий', 'милица', 'пелагея', 'кедрон', 'тофаня', 'григорий', 'адельфина', 'афраний', 'семплеяр', 'арчибальд', 'амвросий', 'владимир', 'тузабубен', 'гестас', 'вульф', 'никанор', 'иисус', 'гамаленький', 'герберт', 'кесария', 'наташа', 'мельпомена', 'анфис', 'малюта', 'карпов', 'михаил', 'анна', 'пантелей', 'мстислав', 'александр', 'ричард', 'роман', 'марк', 'евгений', 'антоний', 'антонид', 'сарид', 'драмлита', 'савва', 'павел', 'иванушка', 'феодосий', 'массолит', 'сергей', 'венера', 'гессара', 'федя', 'гелла', 'тузбубен', 'ксения', 'рудольф', 'иосиф', 'хеврон', 'клавдий', 'иван', 'груни', 'сашка', 'наталья', 'федора', 'даниль', 'андрей', 'маргарита', 'ариман', 'луиз', 'аркадий', 'воланд', 'тамара', 'иероним', 'настасья', 'панаева', 'прасковья', 'николай', 'максимилиан', 'миша', 'софья', 'матвей', 'дарья',

In [5]:
def Lemm_Name(name):
    tokens=prepare_text_for_lda(name)
    result=tokens[0] if len(tokens)>0 else None
    return result
full_name_list=[]
extractor = NamesExtractor()
matches = extractor(text)
print('ready')
        

ready


In [23]:
full_name=[]
for match in matches:
    full_name.append([match.fact.first,match.fact.middle,match.fact.last])
print('Finish')    

Finish


In [10]:
unique_set=[]
unique_set.append(['иуда','пилат',None])
for match in matches:
    found=False
    for name in unique_set:
        if (match.fact.first or match.fact.middle or match.fact.last):
            un
    if not found:
        unique_set.append([match.fact.first, match.fact.middle, match.fact.last])
    #for full_name in unique_set:

print('Finish')    

NameError: name 'un' is not defined

# Работа с неиспользуемыми словами

In [4]:
custom_useless_words=['из-за','марка','ведь','патриарший','борменталь','филипп','кихот','санчо','антоний','сансон','шарик','мой','такой','другой','ваш','этот','сам','свой','тот','варенуха','степ','маргарита','иван','воланд','коровьев','пилат','николай','какой-то','свой','самый','быть','стать','это','что-то','весь','какой','мочь','никанор','который','ты','никакой','азазело','азазелло','берлиоз']

useless_set=set(custom_useless_words)
for name in Names_set:
    useless_set.add(name)
for name in Fam_set:
    useless_set.add(name)
for name in Lasts_set:
    useless_set.add(name)

In [73]:
#Temporal
from docx import Document
from docx.shared import RGBColor
from docx.shared import Pt
from docx.enum.style import WD_STYLE_TYPE
import pymorphy2

def read_file_to_doc_double(filename):
    count_change=0
    count_symbols_var=0
    with open('Lyrics\\'+filename+'.txt',encoding='utf-8') as f:
        document=Document()
        for line in f:
            was_symbol=0
            if len(line)>1:
                paragraph=document.add_paragraph('')
                prev_word=['',False]
                for word in line.split():
                    prev_word[1]=True if was_symbol>0 else False
                    was_symbol=0
                    run = paragraph.add_run(word+' ')
                    tokens=prepare_text_for_lda(word)
                    p = morph.parse(word)[0]
                    if len(tokens)>0 and (tokens[0] not in useless_set):
                        for symbol in symbols:
                            if tokens[0] in symbol:
                                #run.font.highlight_color=4 if prev_word[1] else 7
                                if prev_word[1]:
                                    run.font.highlight_color=4 
                                    count_symbols_var+=1
                                else:
                                    run.font.highlight_color=7
                                count_change+=1
                                was_symbol+=1
                                break
                        prev_word[0]=tokens[0]                                  
        document.save('Outputs//test_double.docx')
        print(count_change)
        print(count_symbols_var)

def neighbor(pos1,pos2):
    if((pos1=="NOUN") or (pos1=="NPRO")) and ((pos2=="VERB")or(pos2=="ADJF")):
        return True
    if( ((pos1=="VERB") or (pos1=="ADJF")) and ((pos2=="NOUN") or (pos2=="NPRO")) ):
        return True
    return False

def read_file(filename):
    Chapter2_tokenized=[]
    Chapter2_text=[]
    with open('Lyrics\\'+filename+'.txt',encoding='utf-8') as f:
        for line in f:
            Chapter2_text.append(line)
            tokens = prepare_text_for_lda(line)
            Chapter2_tokenized.append(tokens)
    return Chapter2_text,Chapter2_tokenized

def take_symbol_sonik_lines(Chapter_tokenized,Chapter_text):
    count=0
    symbol_lines=[]
    for tokens in Chapter_tokenized:
        limit=0
        for token in tokens:
            if limit>0:
                break
            for symbol in symbols:
                if token in symbol:
                    symbol_lines.append(Chapter_text[count])
                    limit+=1
                    break
        count+=1
    #print(count)
    return symbol_lines


read_file_to_doc_double('Chapter2')


#Chapter_text,Chapter_tokenized=read_file('Chapter2')
#symbol_lines=take_symbol_sonik_lines(Chapter_tokenized,Chapter_text)
#for line in symbol_lines:
    #print(line)

921
133


In [62]:
#Гребанные сонники
def read_file_to_doc_sonic(filename):
    count_change=0
    with open('Lyrics\\'+filename+'.txt',encoding='utf-8') as f:
        document=Document()
        for line in f:
            was_symbol=0
            if len(line)>1:
                paragraph=document.add_paragraph('')
                prev_word=['',False]
                for word in line.split():
                    if was_symbol>0:
                        prev_word[1]=True
                    else:
                        prev_word[1]=False
                    was_symbol=0
                    run = paragraph.add_run(word+' ')
                    tokens=prepare_text_for_lda(word)
                    p = morph.parse(word)[0]
                    if len(tokens)>0:
                        if (tokens[0] not in useless_set):
                            for sonic_symbol in sonic_symbols:
                                if tokens[0] in sonic_symbol:
                                    if prev_word[1]:
                                        run.font.highlight_color=4
                                    count_change+=1
                                    was_symbol+=1
                                    break
                                    
                            prev_word[0]=tokens[0]
                                
        document.save('Outputs//test_Sonic.docx')

        
read_file_to_doc_sonic('Chapter2')


1270


# NGramm project

In [7]:


import pymorphy2


from docx import Document
from docx.shared import RGBColor
from docx.shared import Pt
from docx.enum.style import WD_STYLE_TYPE
import pymorphy2


def check_ngramm(ngramm,decision_value):
    count=0
    res=decision_value*len(ngramm)
    for gramm in ngramm:
        tokens=prepare_text_for_lda(gramm)
        if not(len(tokens)>0 and (tokens[0] not in useless_set)):
            continue
        for symbol in symbols:
            if tokens[0] in symbol:
                count+=1
                break
    if (count>=res)and len(ngramm)>1:
        return True
    else:
        return False


def read_file_to_doc_ngramm(filename,len_ngramm,decision_value):
    count_change=0
    count_symbols_var=0
    prev_value=False
    ngramm=[]
    with open('Lyrics\\'+filename+'.txt',encoding='utf-8') as f:
        document=Document()
        for line in f:
            prev_value=False
            if not len(line)>1:
                continue
            temp_bool=check_ngramm(ngramm,decision_value)
            for gramm in ngramm:
                run = paragraph.add_run(gramm+' ')
                if temp_bool:
                    run.font.highlight_color=4
                else:
                    run.font.highlight_color=None
            ngramm=[]
            paragraph=document.add_paragraph('')
            for word in line.split():
                if len(ngramm)==len_ngramm:
                    temp_bool=check_ngramm(ngramm,decision_value)
                    run = paragraph.add_run(ngramm.pop(0)+' ')
                    if temp_bool or prev_value:
                        run.font.highlight_color=4
                    else:
                        run.font.highlight_color=None
                    prev_value=temp_bool
                ngramm.append(word)
        temp_bool=check_ngramm(ngramm,decision_value)
        for gramm in ngramm:
            run = paragraph.add_run(gramm+' ')
            if temp_bool:
                run.font.highlight_color=4
            else:
                run.font.highlight_color=None
        document.save('Outputs//test_ngramm'+str(len_ngramm)+'.docx')


read_file_to_doc_ngramm('Chapter2',2,0.75)
read_file_to_doc_ngramm('Chapter2',3,0.75)
read_file_to_doc_ngramm('Chapter2',4,0.75)
read_file_to_doc_ngramm('Chapter2',5,0.75)
print('ready')

ready


In [39]:
import csv
import pymorphy2
from docx import Document

separators=['...','?','!','.']
count_change=0
count_symbols_var=0
prev_value=False
ngramm=[]
results=[]
filename='Chapter2'
with open('Lyrics\\'+filename+'.txt',encoding='utf-8') as f:
    document=Document()
    for line in f:
        prev_value=False
        if not len(line)>1:
            continue
        paragraph=document.add_paragraph('')
        stroks=line.split('...')
        temps=[]
        for strok in stroks:
            some=strok.split('?')
            for one in some:
                temps.append(one)
        stroks=temps
        temps=[]
        for strok in stroks:
            some=strok.split('!')
            for one in some:
                temps.append(one)
        stroks=temps
        temps=[]
        for strok in stroks:
            some=strok.split('.')
            for one in some:
                temps.append(one)
        stroks=temps
        temps=[]
        for strok in stroks:
            tokens=prepare_text_for_lda(strok)
            for token in tokens:
                if token in symbols:
                    results.append({token:strok})

for result in results:
    print(result)
#print(results)

print('ready')

{'два': 'В белом плаще с кровавым подбоем, шаркающей кавалерийской походкой, ранним утром четырнадцатого числа весеннего месяца нисана в крытую колоннаду между двумя крыльями дворца ирода великого вышел прокуратор Иудеи Понтий Пилат'}
{'верхний': ' От флигелей в тылу дворца, где расположилась пришедшая с прокуратором в Ершалаим первая когорта двенадцатого молниеносного легиона, заносило дымком в колоннаду через верхнюю площадку сада, и к горьковатому дыму, свидетельствовавшему о том, что кашевары в кентуриях начали готовить обед, примешивался все тот же жирный розовый дух'}
{'голова': ' Попробую не двигать головой»'}
{'болезненный': ' Не удержавшись от болезненной гримасы, прокуратор искоса, бегло проглядел написанное, вернул пергамент секретарю и с трудом проговорил:\n'}
{'гримаса': ' Не удержавшись от болезненной гримасы, прокуратор искоса, бегло проглядел написанное, вернул пергамент секретарю и с трудом проговорил:\n'}
{'балкон': 'И сейчас же с площадки сада под колонны на балкон д

{'власть': '–\xa0В числе прочего я говорил, – рассказывал арестант, – что всякая власть является насилием над людьми и что настанет время, когда не будет власти ни кесарей, ни какой-либо иной власти'}
{'власть': ' Человек перейдет в царство истины и справедливости, где вообще не будет надобна никакая власть'}
{'великий': '–\xa0На свете не было, нет и не будет никогда более великой и прекрасной для людей власти, чем власть императора Тиверия'}
{'власть': '–\xa0На свете не было, нет и не будет никогда более великой и прекрасной для людей власти, чем власть императора Тиверия'}
{'власть': '–\xa0На свете не было, нет и не будет никогда более великой и прекрасной для людей власти, чем власть императора Тиверия'}
{'больной': ' – сорванный и больной голос Пилата разросся'}
{'голос': ' – сорванный и больной голос Пилата разросся'}
{'безумный': '–\xa0И не тебе, безумный преступник, рассуждать о ней'}
{'балкон': ' – тут Пилат вскричал: – Вывести конвой с балкона'}
{'балкон': 'Конвой поднял копья

{'выражение': 'В изысканных выражениях извинившись перед первосвященником, он попросил его присесть на скамью в тени магнолии и обождать, пока он вызовет остальных лиц, нужных для последнего краткого совещания, и отдаст еще одно распоряжение, связанное с казнью'}
{'вежливый': 'Каифа вежливо поклонился, приложив руку к сердцу, и остался в саду, а Пилат вернулся на балкон'}
{'балкон': 'Каифа вежливо поклонился, приложив руку к сердцу, и остался в саду, а Пилат вернулся на балкон'}
{'два': ' Там ожидавшему его секретарю он велел пригласить в сад легата легиона, трибуна когорты, а также двух членов Синедриона и начальника храмовой стражи, ожидавших вызова на следующей нижней террасе сада в круглой беседке с фонтаном'}
{'вызов': ' Там ожидавшему его секретарю он велел пригласить в сад легата легиона, трибуна когорты, а также двух членов Синедриона и начальника храмовой стражи, ожидавших вызова на следующей нижней террасе сада в круглой беседке с фонтаном'}
{'аромат': 'Тут все присутствующие

# Script time?


In [9]:
import pymorphy2


from docx import Document
from docx.shared import RGBColor
from docx.shared import Pt
from docx.enum.style import WD_STYLE_TYPE
import pymorphy2

decision_value=0.75

def check_ngramm(ngramm,decision_value):
    count=0
    res=decision_value*len(ngramm)
    for gramm in ngramm:
        tokens=prepare_text_for_lda(gramm)
        if not(len(tokens)>0 and (tokens[0] not in useless_set)):
            continue
        for symbol in symbols:
            if tokens[0] in symbol:
                count+=1
                break
    if (count>=res)and len(ngramm)>1:
        return True
    else:
        return False
    
def read_file_to_doc_ngramm(filename,len_ngramm,decision_value):
    count_change=0
    count_symbols_var=0
    prev_value=False
    ngramm=[]
    result=[]
    with open('Lyrics\\'+filename+'.txt',encoding='utf-8') as f:
        document=Document()
        for line in f:
            prev_value=False
            if not len(line)>1:
                continue
            temp_bool=check_ngramm(ngramm,decision_value)
            if temp_bool:
                toAdd=''
                for gramm in ngramm:
                    toAdd+=gramm + ' '
                result.append(toAdd)
            for word in line.split():
                if len(ngramm)==len_ngramm:
                    temp_bool=check_ngramm(ngramm,decision_value)
                    if temp_bool or prev_value:
                        toAdd=''
                        for gramm in ngramm:
                            toAdd+=gramm + ' '
                        result.append(toAdd)
                    prev_value=temp_bool
                    ngramm.pop(0)
                ngramm.append(word)   
        temp_bool=check_ngramm(ngramm,decision_value)
        if temp_bool:
            toAdd=''
            for gramm in ngramm:
                toAdd+=gramm + ' '
            result.append(toAdd)
    return result



some=read_file_to_doc_ngramm('Chapter2',3,0.75)
for meow in some:
    print(meow)
#read_file_to_doc_ngramm('Chapter2',3,0.75)
#read_file_to_doc_ngramm('Chapter2',4,0.75)
#read_file_to_doc_ngramm('Chapter2',5,0.75)
#print('ready')

двумя крыльями дворца 
крыльями дворца ирода 
дворца ирода великого 
ирода великого вышел 
запах розового масла, 
розового масла, и 
адской болью головой. 
адской болью головой. 
болью головой. Человек 
головой. Человек со 
храм старой веры 
старой веры и 
новый храм истины. 
храм истины. Сказал 
крылом лица медной 
лица медной статуи 
медной статуи в 
страхом тяжкой кары 
тяжкой кары запрещено 
Синедриона, двух членов 
двух членов его 
великого праздника пасхи. 
великого праздника пасхи. 
праздника пасхи. Итак, 
большим искусством. Брови 
искусством. Брови на 
всадник Золотое Копье! 
всадник Золотое Копье! 
Золотое Копье! – 
пятнами, глаза горели. 
глаза горели. Он, 
стенам сада Ирода 
сада Ирода великого. 
Ирода великого. Этот 
двух членов Синедриона 
членов Синедриона и 
комнате лучи солнца 
лучи солнца и 
вверх правую руку, 
правую руку, и 
честь праздника пасхи 
праздника пасхи одному 
пасхи одному из 
боковым ступеням троих 
ступеням троих со 
белыми тюрбанами лица 
тюрбанами лиц

# Объединение

In [10]:
import pymorphy2


from docx import Document
from docx.shared import RGBColor
from docx.shared import Pt
from docx.enum.style import WD_STYLE_TYPE
import pymorphy2

decision_value=0.75

Unused_chars=[':',',','.','-','\n','?','!']

#Логический разделитель
Logic_Sep=['PREP','CONJ','PRCL']
Logic_AdVerb=['PRTF','PRTS','GRND']
Logic_Adj=['ADJF','ADJS']
verb=['VERB','INFN']
Separators=Logic_Sep+Logic_AdVerb+verb+Logic_Adj

def logic_construct_extractor(string):
    Main_noun=''
    verbs=[]
    temp_string=''
    state=False
    Logical_structure=[]
    for word in string.split():
        f_word=word
        for char in Unused_chars:
            if char in f_word:
                f_word=f_word.replace(char,'')
        p=morph.parse(f_word)[0]
        if p.tag.POS in Separators and len(temp_string)>0 and not state:
            Logical_structure.append(temp_string)
            temp_string=f_word+' '
            state=True
        else:
            temp_string+=f_word+' '
            state=False
    Logical_structure.append(temp_string)
    return Logical_structure

def check_symbol_string(string,decision_value):
    count=0
    ngramm=string.split()
    res=decision_value*len(ngramm)
    for gramm in ngramm:
        tokens=prepare_text_for_lda(gramm)
        if not(len(tokens)>0 and (tokens[0] not in useless_set)):
            continue
        for symbol in symbols:
            if tokens[0] in symbol:
                count+=1
                break
    if (count>=res)and len(ngramm)>1:
        return True
    else:
        return False
    
def check_file_symbol(filename,decision_value):
    count_change=0
    count_symbols_var=0
    prev_value=False
    ngramm=[]
    result={}
    with open('Lyrics\\'+filename+'.txt',encoding='utf-8') as f:
        for line in f:
            if not len(line)>1:
                continue
            for string in line.split('.'):
                logic_structs=logic_construct_extractor(string)
                for logic_struct in logic_structs:
                    if check_symbol_string(logic_struct,decision_value):
                        result[logic_struct]=string
    return result



some=check_file_symbol('Chapter2',0.75)

print('ready')

ready


In [21]:
for one in some:
    for sonic in sonic_symbols:
        stop_b=False
        for word in sonic.split(' '):
            if word in one:
                print(one)
                stop_b=True
                break
        if stop_b:
            break

print('\n')
print('\n')            
for one in some:
    print(one)

между двумя крыльями дворца ирода великого 
розового масла 
розовый запах 
левым глазом 
адской болью головой 
разных людей 
старой веры 
новый храм истины 
одного слова 
оставить на время дворец 
умного человека 
латинский язык 
медной статуи 
Крылья ласточки 
золотой венец; 
солнечного луча 
прекрасной для людей власти 
государственное дело 
тяжкой кары 
большим искусством 
Золотое Копье 
самым стенам сада Ирода великого 
тыльной стороной кисти руки 
львиного хвоста 
морском валу 
правую руку 
в честь праздника пасхи 
ведет к боковым ступеням троих 
греческом языках 
белыми тюрбанами лица 
около десяти часов утра 




между двумя крыльями дворца ирода великого 
розового масла 
розовый запах 
левым глазом 
адской болью головой 
разных людей 
старой веры 
новый храм истины 
одного слова 
оставить на время дворец 
умного человека 
латинский язык 
медной статуи 
Крылья ласточки 
золотой венец; 
солнечного луча 
прекрасной для людей власти 
государственное дело 
тяжкой кары 
большим искус

In [5]:
Logic_Sep=['PREP','CONJ','PRCL']
Logic_AdVerb=['PRTF','PRTS','GRND']
Logic_Adj=['ADJF','ADJS']
verb=['VERB','INFN']
Separators=Logic_Sep+Logic_AdVerb+verb+Logic_Adj

def logic_construct_extractor(string):
    Main_noun=''
    verbs=[]
    temp_string=''
    state=False
    Logical_structure=[]
    for word in string.split():
        f_word=word
        for char in Unused_chars:
            if char in f_word:
                f_word=f_word.replace(char,'')
        p=morph.parse(f_word)[0]
        if p.tag.POS in Separators and len(temp_string)>0 and not state:
            Logical_structure.append(temp_string)
            temp_string=f_word+' '
            state=True
        else:
            temp_string+=f_word+' '
            state=False
    Logical_structure.append(temp_string)
    return Logical_structure

count_change=0
count_symbols_var=0
prev_value=False
ngramm=[]
result={}
with open('Lyrics\\'+'Chapter2'+'.txt',encoding='utf-8') as f:
    for line in f:
        if not len(line)>1:
            continue
        for string in line.split('.'):
            logic_structs=logic_construct_extractor(string)
            print(logic_structs)

['\ufeffПонтий Пилат ']
['В белом плаще ', 'с кровавым подбоем ', 'шаркающей кавалерийской походкой ', 'ранним утром ', 'четырнадцатого числа ', 'весеннего месяца нисана ', 'в крытую колоннаду ', 'между двумя крыльями дворца ирода великого ', 'вышел прокуратор Иудеи Понтий Пилат ']
['']
['Более всего ', 'на свете прокуратор ', 'ненавидел запах ', 'розового масла ', 'и все теперь ', 'предвещало нехороший день ', 'так как запах ', 'этот начал ', 'преследовать прокуратора ', 'с рассвета ']
['Прокуратору ', 'казалось что ', 'розовый запах ', 'источают кипарисы ', 'и пальмы ', 'в саду ', 'что к запаху кожи ', 'и конвоя ', 'примешивается проклятая ', 'розовая струя ']
['От флигелей ', 'в тылу дворца где ', 'расположилась пришедшая ', 'с прокуратором ', 'в Ершалаим ', 'первая когорта ', 'двенадцатого молниеносного легиона ', 'заносило дымком ', 'в колоннаду ', 'через верхнюю площадку сада ', 'и к ', 'горьковатому дыму ', 'свидетельствовавшему о ', 'том что кашевары ', 'в кентуриях ', 'начали 

['Секретарь ', 'поднял свиток ', 'решил пока ', 'что ничего ', 'не записывать ', 'и ничему ', 'не удивляться ']
['']
['– ', 'Сознайся – тихо погречески ', 'спросил Пилат – ты ', 'великий врач ']
['– Нет прокуратор я ', 'не врач – ', 'ответил арестант ', 'с наслаждением ', 'потирая измятую ', 'и опухшую ', 'багровую кисть руки ']
['']
['Круто исподлобья Пилат ', 'буравил глазами арестанта ', 'и в ', 'этих глазах уже ', 'не было мути ', 'в них ', 'появились всем ', 'знакомые искры ']
['']
['– Я ', 'не спросил тебя – ', 'сказал Пилат – ты ', 'может быть ', 'знаешь и ', 'латинский язык ']
['– ', 'Да знаю – ', 'ответил арестант ']
['']
['Краска ', 'выступила на ', 'желтоватых щеках Пилата ', 'и он ', 'спросил полатыни ']
['– ', 'Как ты ', 'узнал что я ', 'хотел позвать собаку ']
['– ', 'Это очень ', 'просто – ', 'ответил арестант полатыни – ты ', 'водил рукой ', 'по воздуху – арестант ', 'повторил жест Пилата – ', 'как будто ', 'хотел погладить ', 'и губы ']
['']
['']
['']
['– ', 'Да – ', '

['Ему прокуратор ', 'приказал сдать преступника начальнику тайной службы ', 'и при этом ', 'передать ему распоряжение прокуратора ', 'о том ', 'чтобы Иешуа ', 'ГаНоцри был ', 'отделен от ', 'других осужденных ', 'а также ', 'о том ', 'чтобы команде тайной службы ', 'было под страхом ', 'тяжкой кары ', 'запрещено о ', 'чем бы ', 'то ни ', 'было разговаривать ', 'с Иешуа ', 'или отвечать ', 'на какиелибо его вопросы ']
['']
['По знаку Марка вокруг Иешуа ', 'сомкнулся конвой ', 'и вывел его ', 'с балкона ']
['']
['Затем ', 'перед прокуратором ', 'предстал стройный ', 'светлобородый красавец ', 'со сверкающими ', 'на груди ', 'львиными мордами ', 'с орлиными перьями ', 'на гребне шлема ', 'с золотыми бляшками ', 'на портупее меча ', 'в зашнурованной ', 'до колен обуви ', 'на тройной подошве ', 'в наброшенном ', 'на левое плечо ', 'багряном плаще ']
['Это ', 'был командующий легионом легат ']
['Его прокуратор ', 'спросил о ', 'том где сейчас ', 'находится себастийская когорта ']
['Легат ', 

['Тут ', 'все присутствующие ', 'тронулись вниз ', 'по широкой ', 'мраморной лестнице ', 'меж стен роз ', 'источавших одуряющий аромат ', 'спускаясь все ', 'ниже и ', 'ниже к ', 'дворцовой стене ', 'к воротам ', 'выходящим на ', 'большую гладко ', 'вымощенную площадь ', 'в конце ', 'которой виднелись колонны ', 'и статуи ', 'Ершалаимского ристалища ']
['']
['Лишь ', 'только группа ', 'выйдя из сада ', 'на площадь ', 'поднялась на ', 'обширный царящий ', 'над площадью ', 'каменный помост Пилат ', 'оглядываясь сквозь ', 'прищуренные веки ', 'разобрался в обстановке ']
['То пространство ', 'которое он ', 'только что ', 'прошел то есть пространство ', 'от дворцовой стены ', 'до помоста ', 'было пусто ', 'но зато впереди себя Пилат площади уже ', 'не увидел – ее ', 'съела толпа ']
['Она ', 'залила бы ', 'и самый помост ', 'и то ', 'очищенное пространство ', 'если бы ', 'тройной ряд ', 'себастийских солдат ', 'по левую руку Пилата ', 'и солдат ', 'итурейской вспомогательной когорты ', 'по пр

In [43]:
'''
with open('Outputs\Confirmed_master_90.txt','w',encoding='utf-8') as f:
    for meow in some1:
        f.write('{} \n'.format(meow))
with open('Outputs\Confirmed_master_75.txt','w',encoding='utf-8') as f:
    for meow in some2:
        f.write('{} \n'.format(meow))
with open('Outputs\Confirmed_master_60.txt','w',encoding='utf-8') as f:
    for meow in some3:
        f.write('{} \n'.format(meow))
'''

In [8]:
with open('Outputs\Confirmed.txt','w',encoding='utf-8') as f:
    for meow in some:
        print('Это символ?')
        print('\n')
        print('{}'.format(meow))
        print('\n')
        print('{}'.format(some[meow]))
        if 'y' in (input()).lower():
            f.write('{} \n'.format(meow))
        elif 'q' in (input()).lower():
            break
print('ready')


        
    
    

Это символ?


между двумя крыльями дворца ирода великого 


В белом плаще с кровавым подбоем, шаркающей кавалерийской походкой, ранним утром четырнадцатого числа весеннего месяца нисана в крытую колоннаду между двумя крыльями дворца ирода великого вышел прокуратор Иудеи Понтий Пилат
y
Это символ?


розового масла 


Более всего на свете прокуратор ненавидел запах розового масла, и все теперь предвещало нехороший день, так как запах этот начал преследовать прокуратора с рассвета
y
Это символ?


розовый запах 


 Прокуратору казалось, что розовый запах источают кипарисы и пальмы в саду, что к запаху кожи и конвоя примешивается проклятая розовая струя
q
q
ready


In [ ]:
import random
import glob
text_data = []
semantic_data = []
wn_data_temp=[]
local_dict={}
TO_DELETE=[]
with open('Lyrics\Master.txt',encoding='utf-8') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .001:
            text_data.append(tokens)

2019-03-23 17:38:13,189 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar to C:\Users\nikov\AppData\Local\Temp\tika-server.jar.
2019-03-23 17:38:28,624 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar.md5 to C:\Users\nikov\AppData\Local\Temp\tika-server.jar.md5.
2019-03-23 17:38:29,079 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2019-03-23 17:38:34,082 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


None


In [87]:
##Работа с одним предложением
Unused_chars=[':',',','.','-','\n','?','!']

#Логический разделитель
Logic_Sep=['PREP','CONJ','PRCL']
Logic_AdVerb=['PRTF','PRTS','GRND']
Logic_Adj=['ADJF','ADJS']
verb=['VERB','INFN']
string='В белом плаще с кровавым подбоем, шаркающей кавалерийской походкой, ранним утром четырнадцатого числа весеннего месяца нисана в крытую колоннаду между двумя крыльями дворца ирода великого вышел прокуратор Иудеи Понтий Пилат.'
Logical_structure=[]

Separators=Logic_Sep+Logic_AdVerb+verb+Logic_Adj
Main_noun=''
verbs=[]
temp_string=''
state=False
for word in string.split():
    f_word=word
    for char in Unused_chars:
        if char in f_word:
            f_word=f_word.replace(char,'')
    p=morph.parse(f_word)[0]
    if p.tag.POS in Separators and len(temp_string)>0 and not state:
        Logical_structure.append(temp_string)
        temp_string=f_word+' '
        state=True
    else:
        temp_string+=f_word+' '
        state=False
    if p.tag.POS in verb:
        verbs.append(f_word)
    if p.tag.POS=='NOUN' and p.tag.case=='nomn':
        Main_noun+=f_word+' '
Logical_structure.append(temp_string)
print('Структура?')
print(Logical_structure)
print('\n')

    #print('{} {}'.format(f_word,p.tag.POS))

Кто или что?
прокуратор Иудеи Пилат 


Действие?
['вышел']


Что?
['В белом плаще ', 'с кровавым подбоем ', 'шаркающей кавалерийской походкой ', 'ранним утром ', 'четырнадцатого числа ', 'весеннего месяца нисана ', 'в крытую колоннаду ', 'между двумя крыльями дворца ирода великого ', 'вышел прокуратор Иудеи Понтий Пилат ']




In [80]:
Unused_chars=[':',',','.','-','\n','?','!']
Main_noun=[]
Sup_noun=[]
verbs=[]

#Логический разделитель
Logic_Sep=['PREP','CONJ','PRCL']
Logic_AdVerb=['PRTF','PRTS','GRND']
Logic_Adj=['ADJF','ADJS']
verb=['VERB','INFN']


import pymorphy2
import re
morph = pymorphy2.MorphAnalyzer()
filename='Chapter2'
with open('Lyrics\\'+filename+'.txt',encoding='utf-8') as f:
    for paragraph in f:
        for strok in re.split('[!.?]',paragraph):
            #string=''
            for word in strok.split():
                f_word=word
                for char in Unused_chars:
                    if char in f_word:
                        f_word=f_word.replace(char,'')
                p=morph.parse(f_word)[0]
                print('{} {}'.format(f_word,p.tag.POS))

﻿ None
В PREP
белом NOUN
плаще NOUN
с PREP
кровавым ADJF
подбоем NOUN
шаркающей PRTF
кавалерийской ADJF
походкой NOUN
ранним ADJF
утром ADVB
четырнадцатого ADJF
числа NOUN
весеннего ADJF
месяца NOUN
нисана NOUN
в PREP
крытую ADJF
колоннаду NOUN
между PREP
двумя NUMR
крыльями NOUN
дворца NOUN
ирода NOUN
великого NOUN
вышел VERB
прокуратор NOUN
Иудеи NOUN
Понтий NOUN
Пилат NOUN
Более ADVB
всего ADVB
на PREP
свете NOUN
прокуратор NOUN
ненавидел VERB
запах NOUN
розового ADJF
масла NOUN
и CONJ
все ADJF
теперь ADVB
предвещало VERB
нехороший ADJF
день NOUN
так CONJ
как CONJ
запах NOUN
этот ADJF
начал VERB
преследовать INFN
прокуратора NOUN
с PREP
рассвета NOUN
Прокуратору NOUN
казалось CONJ
что CONJ
розовый ADJF
запах NOUN
источают VERB
кипарисы NOUN
и CONJ
пальмы NOUN
в PREP
саду NOUN
что CONJ
к PREP
запаху NOUN
кожи NOUN
и CONJ
конвоя NOUN
примешивается VERB
проклятая ADJF
розовая ADJF
струя GRND
От PREP
флигелей NOUN
в PREP
тылу NOUN
дворца NOUN
где ADVB
расположилась VERB
пришедшая PRTF
с

мучительно ADJS
вспоминая GRND
зачем ADVB
на PREP
утреннем ADJF
безжалостном ADJF
Ершалаимском ADJF
солнцепеке NOUN
стоит VERB
перед PREP
ним NPRO
арестант NOUN
с PREP
обезображенным PRTF
побоями NOUN
лицом NOUN
и CONJ
какие ADJF
еще PRCL
никому NPRO
не PRCL
нужные ADJF
вопросы NOUN
ему NPRO
придется VERB
задавать INFN
– None
Левий NOUN
Матвей NOUN
– None
хриплым ADJF
голосом NOUN
спросил VERB
больной ADJF
и CONJ
закрыл VERB
глаза NOUN
– None
Да PRCL
Левий NOUN
Матвей NOUN
– None
донесся VERB
до PREP
него NPRO
высокий ADJF
мучающий PRTF
его NPRO
голос NOUN
– None
А CONJ
вот PRCL
что CONJ
ты NPRO
всетаки NOUN
говорил VERB
про PREP
храм NOUN
толпе NOUN
на PREP
базаре NOUN
Голос NOUN
отвечавшего PRTF
казалось CONJ
колол VERB
Пилату NOUN
в PREP
висок NOUN
был VERB
невыразимо ADJS
мучителен ADJS
и CONJ
этот ADJF
голос NOUN
говорил VERB
– None
Я NPRO
игемон NOUN
говорил VERB
о PREP
том ADJF
что CONJ
рухнет VERB
храм NOUN
старой ADJF
веры NOUN
и CONJ
создастся VERB
новый ADJF
храм NOUN
истины

только PRCL
кожа NOUN
его NPRO
утратила VERB
желтизну NOUN
побурела ADJS
а CONJ
глаза NOUN
как CONJ
будто CONJ
провалились VERB
Опятьтаки NOUN
виновата ADJS
была VERB
вероятно CONJ
кровь NOUN
прилившая PRTF
к PREP
вискам NOUN
и CONJ
застучавшая PRTF
в PREP
них NPRO
только PRCL
у PREP
прокуратора NOUN
чтото PRTS
случилось VERB
со PREP
зрением NOUN
Так CONJ
померещилось VERB
ему NPRO
что CONJ
голова NOUN
арестанта NOUN
уплыла VERB
кудато NOUN
а CONJ
вместо PREP
нее NOUN
появилась VERB
другая ADJF
На PREP
этой ADJF
плешивой ADJF
голове NOUN
сидел VERB
редкозубый ADJF
золотой ADJF
венец; None
на PREP
лбу NOUN
была VERB
круглая ADJF
язва NOUN
разъедающая PRTF
кожу NOUN
и CONJ
смазанная PRTF
мазью; None
запавший PRTF
беззубый ADJF
рот NOUN
с PREP
отвисшей PRTF
нижней ADJF
капризною ADJF
губой NOUN
Пилату NOUN
показалось VERB
что CONJ
исчезли VERB
розовые ADJF
колонны NOUN
балкона NOUN
и CONJ
кровли NOUN
Ершалаима NOUN
вдали ADVB
внизу ADVB
за PREP
садом NOUN
и CONJ
все ADJF
утонуло VERB
вокр

террасы NOUN
дворцового ADJF
сада NOUN
от PREP
городской ADJF
площади NOUN
низкое ADJF
ворчание NOUN
над PREP
которым ADJF
взмывали VERB
по PREP
временам NOUN
слабенькие ADJF
тонкие ADJF
не PRCL
то CONJ
стоны NOUN
не PRCL
то CONJ
крики NOUN
Прокуратор NOUN
понял VERB
что CONJ
там ADVB
на PREP
площади NOUN
уже ADVB
собралась VERB
несметная ADJF
толпа NOUN
взволнованных PRTF
последними ADJF
беспорядками NOUN
жителей NOUN
Ершалаима NOUN
что CONJ
эта ADJF
толпа NOUN
в PREP
нетерпении NOUN
ожидает VERB
вынесения NOUN
приговора NOUN
и CONJ
что CONJ
в PREP
ней NPRO
кричат VERB
беспокойные ADJF
продавцы NOUN
воды NOUN
Прокуратор NOUN
начал VERB
с PREP
того ADJF
что CONJ
пригласил VERB
первосвященника NOUN
на PREP
балкон NOUN
с PREP
тем CONJ
чтобы CONJ
укрыться INFN
от PREP
безжалостного ADJF
зноя NOUN
но CONJ
Каифа NOUN
вежливо ADJS
извинился VERB
и CONJ
объяснил VERB
что CONJ
сделать INFN
этого NPRO
не PRCL
может VERB
Пилат NOUN
накинул VERB
капюшон NOUN
на PREP
свою ADJF
чуть ADVB
лысеющую P

– None
это PRCL
не PRCL
оттого ADVB
что CONJ
душно ADJS
а CONJ
тесно ADVB
мне NPRO
стало VERB
с PREP
тобой NPRO
Каифа NOUN
– None
и CONJ
сузив GRND
глаза NOUN
Пилат NOUN
улыбнулся VERB
и CONJ
добавил VERB
– None
Побереги VERB
себя NPRO
первосвященник NOUN
Темные ADJF
глаза NOUN
первосвященника NOUN
блеснули VERB
и CONJ
не PRCL
хуже COMP
чем CONJ
ранее ADVB
прокуратор NOUN
он NPRO
выразил VERB
на PREP
своем ADJF
лице NOUN
удивление NOUN
– None
Что CONJ
слышу VERB
я NPRO
прокуратор NOUN
– None
гордо ADVB
и CONJ
спокойно ADVB
ответил VERB
Каифа NOUN
– None
ты NPRO
угрожаешь VERB
мне NPRO
после PREP
вынесенного PRTF
приговора NOUN
утвержденного PRTF
тобою NPRO
самим ADJF
Может VERB
ли PRCL
это PRCL
быть INFN
Мы NPRO
привыкли VERB
к PREP
тому ADJF
что CONJ
римский ADJF
прокуратор NOUN
выбирает VERB
слова NOUN
прежде ADVB
чем CONJ
чтонибудь VERB
сказать INFN
Не PRCL
услышал VERB
бы PRCL
нас NPRO
ктонибудь VERB
игемон NOUN
Пилат NOUN
мертвыми NOUN
глазами NOUN
посмотрел VERB
на PREP
первосвящ

нижней ADJF
террасе NOUN
сада NOUN
в PREP
круглой ADJF
беседке NOUN
с PREP
фонтаном NOUN
К PREP
этому NPRO
Пилат NOUN
добавил VERB
что CONJ
он NPRO
тотчас ADVB
выйдет VERB
и CONJ
сам ADJF
и CONJ
удалился VERB
внутрь ADVB
дворца NOUN
Пока ADVB
секретарь NOUN
собирал VERB
совещание NOUN
прокуратор NOUN
в PREP
затененной PRTF
от PREP
солнца NOUN
темными ADJF
шторами NOUN
комнате NOUN
имел VERB
свидание NOUN
с PREP
какимто ADJS
человеком NOUN
лицо NOUN
которого ADJF
было VERB
наполовину ADVB
прикрыто PRTS
капюшоном NOUN
хотя CONJ
в PREP
комнате NOUN
лучи NOUN
солнца NOUN
и CONJ
не PRCL
могли VERB
его NPRO
беспокоить INFN
Свидание NOUN
это PRCL
было VERB
чрезвычайно ADVB
кратко ADVB
Прокуратор NOUN
тихо ADVB
сказал VERB
человеку NOUN
несколько ADVB
слов NOUN
после PREP
чего NPRO
тот ADJF
удалился VERB
а CONJ
Пилат NOUN
через PREP
колоннаду NOUN
прошел VERB
в PREP
сад NOUN
Там ADVB
в PREP
присутствии NOUN
всех ADJF
кого NPRO
он NPRO
желал VERB
видеть INFN
прокуратор NOUN
торжественно ADVB
и 

In [3]:
import math

print(math.floor(5*0.75))
print(math.floor(4*0.75))
print(math.floor(3*0.75))
print(math.floor(6*0.75))

3
3
2
4


In [109]:
#Вторая стадия
def csv_read(filename):
    with open('Data\\'+filename+'.csv') as csvfile:
        input_file1 = csv.DictReader(csvfile,delimiter=";")
        input_file=list(input_file1)
        result=[]
        for d in input_file:
            if d['Суперслово/словосочетание']!='' and d['Значение']!='':
                result.append(d)
            else:
                break
        return result
    
input_file=csv_read('Chapter2')
for row in input_file:
    print(row)
print(len(input_file))

OrderedDict([('#', '1'), ('Фрагмент', 'Домик, который был размером в горошину, разросся и стал как спичечная коробка'), ('Суперслово/словосочетание', 'Домик, который был размером в горошину'), ('Характер слова', 'сравнение'), ('Значение', 'очень маленький дом'), ('Дополнительно', '')])
OrderedDict([('#', '2'), ('Фрагмент', 'Так говорила Маргарита, идя с мастером по направлению к вечному их дому, и мастеру казалось, что слова Маргариты струятся так же, как струился и шептал оставленный позади ручей, и память мастера, беспокойная, исколотая иглами память стала потухать. Кто-то отпускал на свободу мастера, как сам он только что отпустил им созданного героя. Этот герой ушел в бездну, ушел безвозвратно, прощенный в ночь на воскресенье сын короля-звездочета, жестокий пятый прокуратор Иудеи, всадник Понтий Пилат.'), ('Суперслово/словосочетание', 'вечный дом'), ('Характер слова', 'символ'), ('Значение', 'могила, рай, ад, ...'), ('Дополнительно', '')])
OrderedDict([('#', '3'), ('Фрагмент', 'Слу

OrderedDict([('#', ''), ('Фрагмент', 'Лицо Пилата исказилось судорогой, он обратил к Иешуа воспаленные, в красных жилках белки глаз и сказал:\n\n? Ты полагаешь, несчастный, что римский прокуратор отпустит человека, говорившего то, что говорил ты? О, боги, боги! Или ты думаешь, что я готов занять твое место? Я твоих мыслей не разделяю! И слушай меня: если с этой минуты ты произнесешь хотя бы одно слово, заговоришь с кем-нибудь, берегись меня! Повторяю тебе: берегись.'), ('Суперслово/словосочетание', 'Лицо исказилось судорогой                       воспаленные, в красных жилках белки глаз'), ('Характер слова', 'аллегории'), ('Значение', 'внутренняя искажённость, несоответствие межды желаемым и вынужденным действием                                     тяжкие сети жестокости'), ('Дополнительно', '')])
OrderedDict([('#', ''), ('Фрагмент', '? Молчать! ? вскричал Пилат и бешеным взором проводил ласточку, опять впорхнувшую на балкон. ? Ко мне! ? крикнул Пилат.'), ('Суперслово/словосочетание', 

In [ ]:
for filename in glob.glob('Data//*.csv'):
    print(filename)
    with open(filename) as csvfile:
        input_file1 = csv.DictReader(csvfile,delimiter=";")
        input_file=list(input_file1)

In [53]:
for row in input_file:
    for key in row:
        print(key)
    break

#
Фрагмент
Суперслово/словосочетание
Характер слова
Значение
Дополнительно


In [72]:
#legacy
count=0
found_symbols=[]
limit=0
for row in input_file:
    limit+=1
    print('Обработано {} из {}'.format(limit,len(symbols)))
    line=row['Суперслово/словосочетание']
    tokens=prepare_text_for_lda(line)
    for symbol in symbols:
        if symbol in tokens:
            add_list={}
            count+=1
            #found_symbols.append('{}   {} '.format(symbol,row['Суперслово/словосочетание']))
            add_list['символ']=symbol
            add_list['словосочетание']=row['Суперслово/словосочетание']
            add_list['фрагмент']=row['Фрагмент']
            add_list['значение']=row['Значение']
            found_symbols.append(add_list)
            break
            #print('Found')
print(count)
print('ready')

Обработано 1 из 5774
Обработано 2 из 5774
Обработано 3 из 5774
Обработано 4 из 5774
Обработано 5 из 5774
Обработано 6 из 5774
Обработано 7 из 5774
Обработано 8 из 5774
Обработано 9 из 5774
Обработано 10 из 5774
Обработано 11 из 5774
Обработано 12 из 5774
Обработано 13 из 5774
Обработано 14 из 5774
Обработано 15 из 5774
Обработано 16 из 5774
Обработано 17 из 5774
Обработано 18 из 5774
Обработано 19 из 5774
Обработано 20 из 5774
Обработано 21 из 5774
Обработано 22 из 5774
Обработано 23 из 5774
Обработано 24 из 5774
Обработано 25 из 5774
Обработано 26 из 5774
Обработано 27 из 5774
Обработано 28 из 5774
Обработано 29 из 5774
Обработано 30 из 5774
Обработано 31 из 5774
Обработано 32 из 5774
Обработано 33 из 5774
Обработано 34 из 5774
Обработано 35 из 5774
Обработано 36 из 5774
Обработано 37 из 5774
Обработано 38 из 5774
Обработано 39 из 5774
Обработано 40 из 5774
Обработано 41 из 5774
Обработано 42 из 5774
Обработано 43 из 5774
Обработано 44 из 5774
Обработано 45 из 5774
Обработано 46 из 57

Обработано 717 из 5774
Обработано 718 из 5774
Обработано 719 из 5774
Обработано 720 из 5774
Обработано 721 из 5774
Обработано 722 из 5774
Обработано 723 из 5774
Обработано 724 из 5774
Обработано 725 из 5774
Обработано 726 из 5774
Обработано 727 из 5774
Обработано 728 из 5774
Обработано 729 из 5774
Обработано 730 из 5774
Обработано 731 из 5774
Обработано 732 из 5774
Обработано 733 из 5774
Обработано 734 из 5774
Обработано 735 из 5774
Обработано 736 из 5774
Обработано 737 из 5774
Обработано 738 из 5774
Обработано 739 из 5774
Обработано 740 из 5774
Обработано 741 из 5774
Обработано 742 из 5774
Обработано 743 из 5774
Обработано 744 из 5774
Обработано 745 из 5774
Обработано 746 из 5774
Обработано 747 из 5774
Обработано 748 из 5774
Обработано 749 из 5774
Обработано 750 из 5774
Обработано 751 из 5774
Обработано 752 из 5774
Обработано 753 из 5774
Обработано 754 из 5774
Обработано 755 из 5774
Обработано 756 из 5774
Обработано 757 из 5774
Обработано 758 из 5774
Обработано 759 из 5774
Обработано 

In [82]:
#print(found_symbols)
#set_symbols=set(found_symbols)
#for symbol in set_symbols:
    #print(symbol)
    #print('\n')
#for symbol in found_symbols:
    #print(symbol)
print(len(found_symbols))

46


In [170]:
from docx import Document
from docx.shared import RGBColor
from docx.shared import Pt
from docx.enum.style import WD_STYLE_TYPE
import pymorphy2

custom_useless_words=['патриарший','борменталь','филипп','кихот','санчо','антоний','сансон','шарик','мой','такой','другой','ваш','этот','сам','свой','тот','варенуха','степ','маргарита','иван','воланд','коровьев','пилат','николай','какой-то','свой','самый','быть','стать','это','что-то','весь','какой','мочь','никанор','который','ты','никакой','азазело','азазелло','берлиоз']

#custom_useless_words=['патриарший']
def read_file_to_doc(filename):
    count_change=0
    with open('Lyrics\\'+filename+'.txt',encoding='utf-8') as f:
        document=Document()
        for line in f:
            if len(line)>1:
                paragraph=document.add_paragraph('')
                for word in line.split():
                    run = paragraph.add_run(word+' ')
                    tokens=prepare_text_for_lda(word)
                    p = morph.parse(word)[0]
                    if len(tokens)>0:
                        if (tokens[0] not in Names_set and 
                        tokens[0] not in Fam_set and 
                        tokens[0] not in Lasts_set and 
                        tokens[0] not in custom_useless_words):
                            for symbol in symbols:
                                if tokens[0]==symbol:
                                    run.font.highlight_color=7
                                    count_change+=1
                                    break
                                
        document.save('Outputs//test1.docx')
        print(count_change)

read_file_to_doc('Master')

11687


In [66]:
Chapter2_tokenized=[]
Chapter2_text=[]
with open('Lyrics\Chapter2.txt',encoding='utf-8') as f:
    for line in f:
        Chapter2_text.append(line)
        tokens = prepare_text_for_lda(line)
        Chapter2_tokenized.append(tokens)

In [67]:
print(len(Chapter2_tokenized))

524


In [74]:
import os
count=0
symbol_lines=[]
for tokens in Chapter2_tokenized:
    for symbol in symbols:
        if symbol in tokens:
            symbol_lines.append(Chapter2_text[count])
    count+=1
    print(count)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [2]:
#model = BERT_CLASS.from_pretrained(PRE_TRAINED_MODEL_NAME_OR_PATH, cache_dir=None)
#model = BERT_CLASS.from_pretrained(bert-base-multilingual-cased, cache_dir='Cache_Dir')

#Утилита по символам.
#


NameError: name 'glob' is not defined